In [46]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data

In [47]:
mnist = input_data.read_data_sets('/tmp/data/', one_hot=True)

Successfully downloaded train-images-idx3-ubyte.gz 9912422 bytes.
Extracting /tmp/data/train-images-idx3-ubyte.gz
Successfully downloaded train-labels-idx1-ubyte.gz 28881 bytes.
Extracting /tmp/data/train-labels-idx1-ubyte.gz
Successfully downloaded t10k-images-idx3-ubyte.gz 1648877 bytes.
Extracting /tmp/data/t10k-images-idx3-ubyte.gz
Successfully downloaded t10k-labels-idx1-ubyte.gz 4542 bytes.
Extracting /tmp/data/t10k-labels-idx1-ubyte.gz


In [48]:
n_nodes_hl1 = 500
n_nodes_hl2 = 500
n_nodes_hl3 = 500

n_classes = 10
batch_size = 100

In [50]:
# height x width
x = tf.placeholder('float', [None, 784])
y = tf.placeholder('float')

In [52]:
def neural_network_model(data):
    
    
    hidden_1_layer = {'weights': tf.Variable(tf.random_normal([748, n_nodes_hl1])),
                      'bias': tf.Variable(tf.random_normal(n_nodes_hl1))}
    hidden_2_layer = {'weights': tf.Variable(tf.random_normal([n_nodes_hl1, n_nodes_hl2])),
                      'bias': tf.Variable(tf.random_normal(n_nodes_hl2))}
    hidden_3_layer = {'weights': tf.Variable(tf.random_normal([n_nodes_hl2, n_nodes_hl3])),
                      'bias': tf.Variable(tf.random_normal(n_nodes_hl3))}
    output_layer = {'weights': tf.Variable(tf.random_normal([n_nodes_hl3, n_classes])),
                      'bias': tf.Variable(tf.random_normal(n_classes))}
    
    l1 = tf.add(tf.matmul(data, hidden_1_layer['weights']) + hidden_1_layer['bias'])
    l1 = tf.nn.relu(l1)
    
    l2 = tf.add(tf.matmul(l1, hidden_2_layer['weights']) + hidden_2_layer['bias'])
    l2 = tf.nn.relu(l2)
    
    l3 = tf.add(tf.matmul(l2, hidden_3_layer['weights']) + hidden_3_layer['bias'])
    l3 = tf.nn.relu(l2)
    
    output = tf.matmul(tf.matmul(l3, output_layer['weights']) + output_layer['bias'])
    
    return output
    

In [ ]:
def train_neural_network(x):
    prediction = neural_network_model(x)
    cost = tf.reduce_mean( tf.nn.softmax_cross_entropy_with_logits(prediction, y) )
    
    optimizer = tf.train.AdamOptimizer().minimize(cost) # learning rate = 0.001